<center>

## MAGICIAN: Mining and gAmblinG servIces sCrapIng and ANalysis

Matteo Martini 636694

In [1]:
mining_pool_url = "https://en.bitcoin.it/wiki/DeepBit9"


#Individuare nel DataSet tutte le transazioni relative a Deepbit.net,ovvero le transazioni che presentano almeno un indirizzo di input o di output appartenente a Deepbit_addrs.
#servizio scommesse = DiceOnCrack

## Setup Ambiente

In [2]:
# Librerie concesse
import pandas as pd

# Per scraping (scegline uno o entrambi, in base alle esigenze delle pagine)
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

# Per formati efficienti (PyArrow)
import pyarrow as pa
import pyarrow.parquet as pq

# Utility
from pathlib import Path
from datetime import datetime

import numpy as np


from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys
import time
import requests
import re
from bs4 import BeautifulSoup

# Percorsi (aggiorna con il tuo percorso locale)
DATA_DIR = Path('./Datasets')   # metti qui i CSV scaricati

PARQUET_DIR = Path('./data_parquet')
PARQUET_DIR.mkdir(parents=True, exist_ok=True)
print(DATA_DIR)
print(PARQUET_DIR)
print('Environment ready.')

Datasets
data_parquet
Environment ready.


## 1) Costruzione dei Dataframe dal Dataset

## Directory Datasets


In [3]:
transactions_csv = DATA_DIR / 'transactions.csv'
inputs_csv       = DATA_DIR / 'inputs.csv'
outputs_csv      = DATA_DIR / 'outputs.csv'
mapping_csv      = DATA_DIR / 'mapAddr2Ids8708820.csv'

## Definizone Data Type
Sapendo cosa contengono i file csv, possiamo forzare la conversione in determiinati tipi per ridurre l'impatto in memoria dei dataset

In [42]:
dtype_transactions = {
    "timestamp": np.int32,          # prima colonna: secondi (1231469xxx) rientra in int32
    "tx_id": np.int32,              # seconda colonna: contatore 1…10 572 827
    "block_height": np.int32,       # terza colonna: 1…214 562
    "prev_tx_id": np.int32,         # quarta colonna: seguono il tx_id precedente
    "has_witness": np.uint8,        # quinta colonna 0/1
    "fee": np.int32,                # sesta colonna: 0…100 000 (int32 più che sufficiente)
}


dtype_inputs = {
    "input_id": np.int32,
    "tx_id": np.int32,
    "prev_output_id": np.int32,
    "input_index": np.uint8,
}


dtype_outputs = {
    "col0":np.int32,
    "col1":np.int32,
    "col2":np.int8,
    "col3":np.int32,
    "col4":np.int64,
    "col5":"uint8"
}

dtype_mapping = {
    "address": "category",   # stringhe ripetute → category riduce drasticamente
    "address_id": "Int32",   # valori 0–8 708 820 entrano in 32 bit
}

columns_transactions = [
    "timestamp", "tx_id", "block_height",
     "has_witness", "fee"
]

## Caricamento dei Dataset

In [43]:
transactions_csv = DATA_DIR / 'transactions.csv'
inputs_csv       = DATA_DIR / 'inputs.csv'
outputs_csv      = DATA_DIR / 'outputs.csv'
mapping_csv      = DATA_DIR / 'mapAddr2Ids8708820.csv'

transactions = pd.read_csv(transactions_csv,header=None,names=columns_transactions,dtype=dtype_transactions)
inputs_df    = pd.read_csv(inputs_csv,dtype=dtype_inputs)
outputs_df   = pd.read_csv(outputs_csv,dtype=dtype_outputs)
mapping_df   = pd.read_csv(mapping_csv,dtype=dtype_mapping)

## 2) Recupero informazioni da WalletExplorer


### Definizione Funzioni Helper per Scraping

In [ ]:

#placeholder
wallet_explorer_url = "https://www.walletexplorer.com/"

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/127.0.0.1 Safari/537.36"
    ),
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Referer": "https://www.imdb.com/",
}

In [ ]:
def format_transactions(transactions):
    page_transactions = []
    for trans in transactions:
        timestamp = trans.find('td',class_="date").get_text()
        balance = trans.find('td',class_="amount").get_text()
        txid = trans.find('td',class_="txid").find('a')['href']
        match = re.search(r"/txid/([0-9a-f]+)", txid)
        if match:
            txid = match.group(1)
            
        amount_diff = trans.find('td',class_="amount diff")
        page_transactions.append({
            'timestamp':timestamp,
            'balance':balance,
            'txid':txid,
            'amount diff':amount_diff
        })
    return page_transactions

def get_class_transactions(soup,class_):
    transactions = soup.find_all('tr',class_=class_)
    class_trans = format_transactions(transactions)
    # print(f'found {len(class_trans)} {class_} transactions')
    return class_trans

def get_page_transactions(url):
    res = requests.get(f'{url}',headers=headers)
    soup = BeautifulSoup(res.text,"html.parser")
    global_trans = get_class_transactions(soup,'sent') + get_class_transactions(soup,'received')

    return global_trans


### Definizione Funzione per Scraping

In [ ]:
def next_page(driver):
    try:
        #cerco il link alla prossima pagina
        el = driver.find_element(By.XPATH, '//div[@class="paging"]//a[contains(., "Next")]')
        url = el.get_attribute("href")
        el.click()
        return url
    except:
        print("fondo")
        return None

def scrape_step(driver,url):
    #recupero le entries
    page_entries = get_page_transactions(url)
    #vado alla prossima pagina
    url = next_page(driver)
    return [url,page_entries]

def scrape_wallet_explorer(wallet_name):
    driver = webdriver.Chrome()
    driver.get(wallet_explorer_url)
    
    #ricerca del wallet
    driver.find_element(By.NAME,'q').send_keys(wallet_name)
    driver.find_element(By.TAG_NAME,'form').submit()

    url = driver.current_url
    i = 0
    all_entries = []
    while True:
        #recupero di tutte le transazioni della pagina
        url,page_entries = scrape_step(driver,url)
        all_entries.extend(page_entries)
        if url == None:
            break;
    driver.close()
    return all_entries

#https://www.walletexplorer.com/txid/d71d3b18df328681c4e0e4ed215a19820c7ecff956e1b6961eec10b7b3417917


In [31]:
deepbit_transaction = scrape_wallet_explorer('BW')

#1.2296645641326904
#0.6821258068084717
#BW.com

fondo


In [44]:
deepbit_df = pd.DataFrame(deepbit_transaction)
display(deepbit_df.head(10))
transactions.head(1)

,timestamp,balance,txid,amount diff
0,2016-02-07 10:11:34,-400.,b71b5fd9224103b25d6f6ad2b28aae703312db9b57a113...,[-400. ]
1,2016-02-06 17:18:27,-400.,8122f2f88694d6f71330eb57df02cadbf731b412554a03...,[-400. ]
2,2016-02-05 17:16:39,-400.,47fb721c713631ff77462ff1b4e7ca4ee487bb7bbe7ff1...,[-400. ]
3,2016-02-04 18:39:21,-400.,00b9a1326d1d0a1de88c48d654747b4b6baadfd1b501ef...,[-400. ]
4,2016-02-04 03:58:15,-400.,0e2d10d8aa01660548e50c5ee7eb51589ab5bd5a2f2f51...,[-400. ]
5,2016-02-03 03:02:17,-400.,7443375439f6e2eff47e9944d7579a3f7faecd0bc01727...,[-400. ]
6,2016-02-07 20:41:24,+25.06844012,c0996b24787df4bc443e2ef5c0c98a2206e2c8c7496437...,[+25.06844012 ]
7,2016-02-07 19:56:25,+25.08928942,983eb1592501684c2e07d0cf28f067ec9c592f6cf6cc91...,[+25.08928942 ]
8,2016-02-07 19:26:02,+25.23670351,e8f9eae08cbe6a299991613fb98eb82b42262cac851ebd...,[+25.23670351 ]
9,2016-02-07 18:29:25,+25.29877219,b0260cdadf823637b80cf8258869f19a3b9e4ff7057480...,[+25.29877219 ]


,timestamp,tx_id,block_height,has_witness,fee
0,1231006505,0,0,1,0


In [32]:
print(deepbit_transaction[0]['txid'])
print(len(deepbit_transaction[0]['txid']))
#100 entries per pagina
print(len(deepbit_transaction))

b71b5fd9224103b25d6f6ad2b28aae703312db9b57a1136f225a05b907848e13
64
5100
